In [1]:
%qtconsole

In [2]:
# the required python libraries imported
import bnpy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
from bokeh.core.validation import silence
from bokeh.core.validation.warnings import MISSING_RENDERERS
from bokeh.layouts import column
from IPython.core.display import display, HTML
import bokeh
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [3]:
# indicates to jupyer how the plots are to be displayed and sized and some other
# housekeeping particular to this notebook
display(HTML("<style>div.output_scroll { height: 600em; }</style>"))
silence(MISSING_RENDERERS, True)
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 3]

# Theses values need to make sense, mod(data_init_size, batch_size) == 0
data_start = 0
data_init_size = 20000
batch_size = 2000
batchnum = int(data_init_size/batch_size)

# all_data contains all the data
all_data = pd.read_csv('../data/anomaly0245.csv')
all_data.drop(all_data.columns[0], inplace=True, axis=1)

# init_data contains the initialization data
init_data = all_data.head(data_init_size)
init_data = bnpy.data.XData.from_dataframe(init_data)

data_set = bnpy.data.XData.from_dataframe(all_data)

batches = []

i = 0
while i < len(all_data)- batch_size:
    df = all_data.iloc[i:i + batch_size]
    batches.append(bnpy.data.XData.from_dataframe(df))
    i += batch_size

# Graph the data for inspection if required
# p = figure(title="Streaming Data", x_axis_label='x', y_axis_label='y', plot_height=350, plot_width=1200)
# add a line renderer with legend and line thickness
# p.line(all_data.index.tolist(), all_data['anomaly'].tolist(), legend_label="Temp.", line_width=2)
# show the results
# show(p)

In [4]:
# Setup the placekeeping and initilizing variables
chain = 0
x, eng_val, states, num_states = [], [], [], []
step = 5000

# Initialize bnpy model and do initial training
# *DiagGauss* observation model
gamma = 1.0
sF = 1.0
K = 25  # Initialize K component - this value places a max K the model can develop
nLap = 10

# cold start the model
cold_start_model, cold_info_dict = bnpy.run(
    init_data, 'DPMixtureModel', 'DiagGauss', 'memoVB',
    output_path='/tmp/AsteriskK8/coldstart-K=10/',
    nLap=nLap, nTask=1, nBatch=batchnum, convergeThr=0.0001,
    gamma0=gamma, sF=sF, ECovMat='eye',
    K=K, 
    moves='birth,merge,delete,shuffle',
    initname='randexamplesbydist', ts=True, debug=False)

# Initialize graphing vars for first pass with no data 
y = []
x = []
x_batches = []
x_batch_post = []
x_batch_pre = []
K_model = []
K_states = []
index = []

# Make this switch for first pass of warm start loop
warm_start_model = cold_start_model
warm_info_dict = cold_info_dict

  --ts
Dataset Summary:
X Data
  total size: 20000 units
  batch size: 2000 units
  num. batches: 10
Allocation Model:  DP mixture with K=0. Concentration gamma0= 1.00
Obs. Data  Model:  Gaussian with diagonal covariance.
Obs. Data  Prior:  independent Gauss-Wishart prior on each dimension
  Wishart params 
    nu = 3 
  beta = [ 1] 
  Expectations
  E[  mean[k]] = 
  [ 0]
  E[ covar[k]] = 
  [[1.]]
Initialization:
  initname = randexamplesbydist
  K = 25 (number of clusters)
  seed = 1607680
  elapsed_time: 0.0 sec
Learn Alg: memoVB | task  1/1 | alg. seed: 1607680 | data order seed: 8541952
task_output_path: /tmp/AsteriskK8/coldstart-K=10/1
MERGE @ lap 1.00: Disabled. Cannot plan merge on first lap. Need valid SS that represent whole dataset.
DELETE @ lap 1.00: Disabled. Cannot delete before first complete lap, because SS that represents whole dataset is required.
    0.100/10 after      0 sec. |    216.5 MiB | K   25 | loss -5.716921801e-01 |  
    0.200/10 after      1 sec. |    21

In [5]:
vars(warm_start_model.obsModel.Post)
# vars(warm_start_model.obsModel.Prior)

{'K': 8,
 'D': 1,
 '_FieldDims': {'nu': 'K', 'kappa': 'K', 'm': ('K', 'D'), 'beta': ('K', 'D')},
 'doCollapseK1': False,
 'nu': array([2.0003e+04, 3.0000e+00, 3.0000e+00, 3.0000e+00, 3.0000e+00,
        3.0000e+00, 3.0000e+00, 3.0000e+00]),
 'kappa': array([2.00000001e+04, 1.00000000e-04, 1.00000000e-04, 1.00000000e-04,
        1.00000000e-04, 1.00000000e-04, 1.00000000e-04, 1.00000000e-04]),
 'm': array([[2.55430891e+01],
        [5.10861784e-31],
        [5.10861784e-31],
        [5.10861784e-31],
        [5.10861784e-31],
        [5.10861784e-31],
        [5.10861784e-31],
        [5.10861784e-31]]),
 'beta': array([[59.53989049],
        [ 1.        ],
        [ 1.        ],
        [ 1.        ],
        [ 1.        ],
        [ 1.        ],
        [ 1.        ],
        [ 1.        ]])}

In [6]:
bokeh.io.reset_output()
bokeh.io.output_notebook()
output_notebook()                
p1 = figure(title="Dataset", plot_height=200, plot_width=800)
p2 = figure(title="Sufficient Statistics", plot_height=100, plot_width=800)
p3 = figure(title="Average K Resp", plot_height=200, plot_width=800)
#p4 = figure(title="K2", plot_height=100, plot_width=800)
#p5 = figure(title="K3", plot_height=100, plot_width=800)
p6 = figure(title="K", plot_height=200, plot_width=800)
p = column(p1, p2, p3, p6)# p3, p4, p5, p6)
target = show(p, notebook_handle=True)

Loading BokehJS ...

Loading BokehJS ...

In [7]:
st_idx = int(data_init_size/batch_size)
ed_idx = int(data_init_size/batch_size) + 200
elapsed = 0
# for i in range(st_idx, len(batches)):
for ii, abatch in enumerate(batches):
    start = time.time()
    LP = warm_start_model.calc_local_params(abatch)
    #K_resp = [LP["resp"][:,0], LP["resp"][:,1], LP["resp"][:,2]]
    K_resp = np.mean(LP["resp"], axis=0)
    # Setup the bokeh plots and render via call back
    line1 = p1.line(x = x, y = y, color='blue', name='g1', line_width=1)
    line2 = p2.scatter(x = K * [index], y = x_batch_post, color='blue', name='g1', line_width=1)
    line4 = p3.scatter(x = K * [index], y = K_resp , color='red', name='g4')
    line7 = p6.scatter(x = index, y = LP['resp'].shape[1] , color='red', name='g4')
    push_notebook(handle = target)

    # Shift the dataset to include new incoming data   
    start_idx = data_start + (ii*batch_size)
    end_idx = data_start + ((ii+1)*batch_size)
    new_dataset = data_set.make_subset(example_id_list = list(range(start_idx, end_idx)))

    # Put together the warm start model - starting from the previous iteration 
    output_path = f'/tmp/AsteriskK8/warmstart-K=10/b{ii}'  # Dynamic output path according to batch
    warm_start_model, warm_info_dict = bnpy.run(
        new_dataset, 'DPMixtureModel', 'DiagGauss', 'memoVB',
        output_path=output_path,
        nLap=nLap, nTask=1, nBatch=batchnum, convergeThr=0.0001,
        gamma0=gamma, sF=sF, ECovMat='eye',
        K=K, 
        moves='birth,merge,delete,shuffle',
        initname=warm_info_dict['task_output_path'],
        ts=True, debug=True)

    # Check sufficient statistics on the new batch with the newly learned model 
    LPanomaly = []
    SSanomaly = []
    LP = warm_start_model.calc_local_params(abatch)
    LPanomaly.append(LP)  # Calculation of responsibility, needed for next step
    SSanomaly.append(warm_start_model.get_global_suff_stats(abatch, LP))  # Calculation of SS for new data
    x_batch_post = []
    xx_batch_post = []
    K_model = []
    K_states = []
    for key in SSanomaly:
        x_batch_post.append(key.x)
        xx_batch_post.append(key.xx)    
        K_model.append(key.K)
    x_batch_post = np.vstack(x_batch_post)
    xx_batch_post = np.vstack(xx_batch_post)
    K_model = np.vstack(K_model)
    
    index = int(ii * batch_size) + batch_size
    x_batch_pre = np.squeeze(np.squeeze(x_batch_pre))
    x_batch_post = np.squeeze(np.squeeze(x_batch_post))
    K_model = np.sum(x_batch_post > 1)
    y = np.squeeze(abatch.X)
    x = list(range(ii*len(abatch.X), ii*len(abatch.X) + len(abatch.X)))
    end = time.time()
    elapsed = end - start

  --ts
Dataset Summary:
X Data
  total size: 400000 units
  batch size: 200 units
  num. batches: 10
Allocation Model:  DP mixture with K=0. Concentration gamma0= 1.00
Obs. Data  Model:  Gaussian with diagonal covariance.
Obs. Data  Prior:  independent Gauss-Wishart prior on each dimension
  Wishart params 
    nu = 3 
  beta = [ 1] 
  Expectations
  E[  mean[k]] = 
  [ 0]
  E[ covar[k]] = 
  [[1.]]
Initialization:
  initname = /tmp/AsteriskK8/coldstart-K=10/1
  K = 25 (number of clusters)
  seed = 1607680
  elapsed_time: 0.0 sec
Learn Alg: memoVB | task  1/1 | alg. seed: 1607680 | data order seed: 8541952
task_output_path: /tmp/AsteriskK8/warmstart-K=10/b0/1
MERGE @ lap 1.00: Disabled. Cannot plan merge on first lap. Need valid SS that represent whole dataset.
DELETE @ lap 1.00: Disabled. Cannot delete before first complete lap, because SS that represents whole dataset is required.


    0.100/10 after      0 sec. |    218.5 MiB | K    8 | loss -9.414603807e-01 |  
    0.200/10 after      0 sec. |    218.5 MiB | K    8 | loss -1.114773089e+00 |  
    0.300/10 after      1 sec. |    218.5 MiB | K    8 | loss -1.224225600e+00 |  
BIRTH @ lap 1.00 : Added 0 states. 0/10 succeeded. 0/10 failed eval phase. 10/10 failed build phase.
    1.000/10 after      2 sec. |    218.5 MiB | K    8 | loss -1.383675961e+00 |  
BIRTH @ lap 2.000 : None attempted. 0 past failures. 0 too small. 8 too busy.
DELETE @ lap 2.00: 0/1 accepted. Ndiff 0.00.
MERGE @ lap 2.00 : 3/3 accepted. Ndiff 0.00. 16 skipped.
    2.000/10 after      2 sec. |    218.5 MiB | K    5 | loss -1.383675961e+00 | Ndiff    0.000 
BIRTH @ lap 3.000 : None attempted. 0 past failures. 0 too small. 5 too busy.
DELETE @ lap 3.00: 1/1 accepted. Ndiff 0.00.
    3.000/10 after      2 sec. |    218.5 MiB | K    4 | loss -1.383675961e+00 | Ndiff    0.000 
BIRTH @ lap 4.000 : None attempted. 0 past failures. 0 too small. 4 to

RuntimeError: Columns need to be 1D (y is not)